# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files("data/song_data")

check that the song_files has already been filled

In [ ]:
len (song_files)

Selecting the first song in the created list (song_files)

In [ ]:
filepath = song_files[0]

Reading the song files and viewing its data to confirm the correct implementation

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

this line is for myself, I needed to check the data type that were used for each c

In [ ]:
df.dtypes

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

I will use here (df.values) to select the values from the created dataframe.

In [ ]:
df.values[0]

Now, I will select the culomns for song id, title, artist id, year, duration from the array

In [ ]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

now, I will convert the array inot a list and index the first record 

In [ ]:
song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

I got error when I first ran cur.execute (song_table_insert, song_data). So, I decided to see the first argument output

In [ ]:
song_table_insert

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
df.values[0]

In [ ]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

In [ ]:
artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

In [ ]:
artist_table_insert

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")

In [ ]:
len(log_files)

In [ ]:
filepath = log_files[0]
filepath

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df['page']== 'NextSong']
#df

In [ ]:
#df = df.reset_index()


In [ ]:
df.head()

In [ ]:
# I will have a look at the (ts) columns to better understand its nature
df.ts

In [ ]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

In [ ]:
#to continue further, I need to import datetime 
import datetime

In [ ]:
year = df['year'] = t.dt.year
month = df['month'] = t.dt.month
day = df['day'] = t.dt.day
week = df['week'] = t.dt.week
weekday = df['week day'] = t.dt.strftime('%A')
hour = df['hour'] = t.dt.hour
#df.head()
#weekday.dtypes
#day.dtypes
#week.dtypes

In [ ]:
time_data = (t, hour , day, weekday, week, month, year)
column_labels = ('start_time', 'Hour', 'Day', 'weekday', 'week','month', 'year')
                 

In [ ]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

In [ ]:
#adding the start_time columns to the already existed dataframe
df['start_time'] = time_df.start_time
df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
df.dtypes

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName' , 'lastName', 'gender', 'level']]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
x =[] #this list will be used to deal with the repetition of userId primary key

for i, row in user_df.iterrows():
    #print(row.userId)
    if row.userId in x:
        #print('666666666666666666')
        continue
        #print('77777777777777777')
    else:
        #print('8888888888888888')
        x.append(row.userId)
    cur.execute(user_table_insert, row)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
song_select

In [ ]:
songplay_table_insert

In [ ]:
df.head()
df.dtypes


In [ ]:

for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row['song'], row['artist'], row['length']))
    results = cur.fetchone()
    #print(type(results))
    #print(results)
    print('-----------fetchone done--------')
    if results:
        print('-----------results true----------')
        songid, artistid = results
        #songid = results.song_id,
        #artistid = results.artist_id
    else:
        songid, artistid = None, None
        print('------------results false-------------')
        #continue

    # insert songplay record
    #setting the songplay_data to have the following columns: 
    # songplay_id, start_time, user_id, level, song_id, artist_id
    # session_id, location, user_agent
    songplay_data = ( row.start_time, row.userId, row.level,songid, artistid, row.sessionId, row.location, row.userAgent)
    try:
        cur.execute(songplay_table_insert, songplay_data)
    except psycopg2.Error as e:
        print ('Error: Inserting Rows')
        print (e)
    #print(songplay_data)
    conn.commit()

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.